TODO somebody should write something.

The graph shows the average number of cases per 100k per week. The canton does not give us exact data[^1], so we do not know the exact values -- but we do know they are within the range shown in light violet.

[^1]: To preserve anonymity, the canton does not give exact numbers, but intervals: we only know that this week there were 0-3 cases, or 4-6, or 7-9...

In [ ]:
# TODO deal with scale properly
SCALE = 680

In [ ]:
### Geo data ##################################

import json

with open('data/plz-ZH-lowres.geojson') as f:
    geo_data = json.load(f)

# copy plz to be used as key for choro_data
for feature in geo_data['features']:
    feature['plz'] = feature['properties']['plz']

In [ ]:
### Covid data ################################

import pandas
import collections
covid_all = pandas.read_excel('data/f6x1.xlsx')

def mk_choro_data(d):
    covid_now = covid_all[covid_all['Date'] == str(d)]
    # display(covid_now)
    return collections.defaultdict(
        lambda: 0,
        zip(covid_now['PLZ'], covid_now['rate7_100K'])
    )

# covid_all

In [ ]:
### Date slider ###############################

import ipywidgets as widgets
import pandas as pd
from datetime import date, timedelta

# TODO when I grow up, I should show the averaging interval, and have a scale underneath

start_date = min(covid_all['Date']).date()  # TODO + 7 or what?
end_date   = max(covid_all['Date']).date()
dates      = pd.date_range(start_date, end_date, freq='D')

def show_date(date):
    fmt = '%b %d'
    interval_start = date - timedelta(days=7)
    return '{} - {}'.format(interval_start.strftime(fmt), date.strftime(fmt))

options = [(show_date(date), date) for date in dates]

date_slider = widgets.SelectionSlider(
    options=options,
    value=end_date,
    description='Date:',
    orientation='horizontal',
    continuous_update=False,
    layout={'width': '400px'},
)

In [ ]:
### Graph ####################################

import bqplot as plt
import numpy as np

my_color = '#4200ad'

date_min = min(covid_all['Date'])
date_max = max(covid_all['Date'])

x_sc = plt.DateScale(min=date_min.date(), max=date_max.date())
# y_sc = plt.LinearScale(min=0, max=max(covid_all['rate7_100K']))
y_sc = plt.LinearScale(min=1, max=(SCALE-1))
scales = {'x': x_sc, 'y': y_sc}

line = plt.Scatter(scales=scales, colors=[my_color],
                   marker='square', default_size=3)

bars = plt.Bars(scales=scales, type='stacked',
                colors=[my_color], opacities=[0,0.4,0], padding=0)

line20 = plt.Lines(x=np.array(['2010', '2030'], dtype='datetime64[D]'), y=np.array([20, 20]), scales=scales,
                   stroke_width=1, colors=['red'])

x_ax = plt.Axis(scale=x_sc, grid_lines='none', num_ticks=2)
y_ax = plt.Axis(scale=y_sc, orientation='vertical', grid_lines='none',
                tick_values=[0,20,100])

index_sel = plt.interacts.IndexSelector(scale=x_sc, marks=[line],
                                        selected=np.array([date_max], dtype='datetime64[D]'),
                                        color='steelblue')

time_plot = plt.Figure(marks=[line20, bars, line], axes=[x_ax,y_ax],
                       interaction=index_sel,
                       padding_y=0, fig_margin={'top': 10, 'right': 25, 'bottom': 25, 'left': 35})
time_plot.layout.height = '200px'
time_plot.layout.width  = '400px'

#index_sel.selected

In [ ]:
# from sidecar import Sidecar
# sc = Sidecar(title='map')

In [ ]:
### Map #######################################

import ipyleaflet


### TODO move these:

help_text = """
<b>Hover over an area</b> to see its data.<br/>
Click to lock onto a specific area.<br/>
Then click the 🔓 unlock button to unlock.
"""

info = widgets.HTML(help_text)
# info.layout.width  = '200px'
info.layout.margin = '0'

unlock_button = widgets.Button(description='🔓', tooltip='Unlock (select by hover instead of click)', disabled=True)
unlock_button.layout.width = 'auto'
unlock_button.layout.right = '0'

In [ ]:
m = ipyleaflet.Map(center=(47.36667, 8.55), zoom=12, zoom_control=False, scroll_wheel_zoom=True)
m.layout.height = '600px'

m.add_control(ipyleaflet.WidgetControl(widget=date_slider, position='bottomleft'))
m.add_control(ipyleaflet.FullScreenControl(position='topright'))
m.add_control(ipyleaflet.ZoomControl(position='topright'))

m.add_control(ipyleaflet.WidgetControl(widget=time_plot, position='bottomleft'))
m.add_control(ipyleaflet.WidgetControl(widget=widgets.HBox([info, unlock_button], layout=widgets.Layout(margin='10px')), position='bottomleft'))


# with sc:
#     display(m)
#     sc.clear_output(wait=True)
display(m)

In [ ]:
from branca.colormap import linear

covid_layer = ipyleaflet.Choropleth(
    name="Covid19 some data ahem TODO",
    geo_data=geo_data,
    choro_data=mk_choro_data(date_slider.value),
    key_on='plz',
    value_min=-10,
    value_max=(SCALE*0.9),
    colormap=linear.Purples_07,
    style_callback=lambda feature, colormap, value: {'fillOpacity': 0.6, 'weight': 2, 'opacity': 0, 'color': '#529', 'fillColor': colormap(value)},
    hover_style={'opacity': 1}
)
m.add_layer(covid_layer)

In [ ]:
### Interactivity ##########################################

plz_search_control = ipyleaflet.SearchControl(
    layer=covid_layer,
    property_name='plz',
    zoom=12,
    position="topleft",
)
m.add_control(plz_search_control)



def on_date_change(change):
    display(change)
    covid_layer.choro_data = mk_choro_data(change['new'])
    
date_slider.observe(on_date_change, names='value')
# date_slider.observe(on_date_change)

    

import traitlets
class FeatureSelector(traitlets.HasTraits):
    selected = traitlets.Any(default_value=None, allow_none=True)
    active   = traitlets.Any(default_value=None, allow_none=True)
    key      = traitlets.Unicode(default_value='id')
    map_layer = traitlets.Instance(klass=ipyleaflet.FeatureGroup)
    selected_style = traitlets.Dict(default_value={'opacity': 1, 'color': 'red'})

    @traitlets.observe('map_layer')
    def _map_layer_changed(self, change):
        self.map_layer.on_click(self.handle_click)
        self.map_layer.on_hover(self.handle_hover)
        self._orig_style_callback = self.map_layer.style_callback
    
    @traitlets.observe('selected')
    def _set_selected_style(self, change):
        def new_style_callback(feature, colormap, value):
            style = self._orig_style_callback(feature, colormap, value) if self._orig_style_callback else {}
            if feature[self.key] == self.selected:
                return {**style, **self.selected_style}
            else:
                return style
        
        self.map_layer.style_callback = new_style_callback
    
    def handle_click(self, feature, **kwargs):
        if self.selected == feature[self.key]:  # deselect
            self.selected = None
        else:
            self.selected = feature[self.key]
            self.active = self.selected

    def handle_hover(self, feature, **kwargs):
        self.active = self.selected if self.selected else feature[self.key]


# TODO names should be only in the spreadsheet, the data files would be more compact that way
feature_names = {f['plz']: f['properties']['ortbez'] for f in geo_data['features']}
def update_info(change, **kwargs):
    plz = change['new']
    info.value = f'''
        <span style="font-size: 1.6em">{plz} {feature_names[plz]}</span><br/>
        <b style="font-size: 2.2em">{covid_layer.choro_data[plz]:.1f}</b><br/>
        cases/week/100k<br/>
        (population: TODO show)
    '''
    
def update_graph(change, **kwargs):
    plz = change['new']
    mine  = covid_all[covid_all['PLZ'] == plz]
    mine0 = mine[mine['rate7_100K'] > 0]
    line.x = np.array(mine0['Date'], dtype='datetime64')
    line.y = np.array(mine0['rate7_100K'])
    bars.x = np.array(mine['Date'], dtype='datetime64')
    bars.y = np.array([mine['rate_min'], mine['rate_max'] - mine['rate_min']])

def unlock_activate(change, **kwargs):
    unlock_button.disabled = not change['new']
    
plz_selector = FeatureSelector(key='plz', map_layer=covid_layer)
plz_selector.observe(update_info,  names='active')
plz_selector.observe(update_graph, names='active')
plz_selector.observe(unlock_activate, names='selected')

def unlock_select(butt):  # it's a butt!
    plz_selector.selected=None

unlock_button.on_click(unlock_select)